In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./year.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.moogparts.com/driv/partfinder/api.catalog.makes?brand=fllj&locale=en_US&country_code=US&year_id=' + input_.loc[a, 'Year_Id'] + '&vehicle_group_ids=2',
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Year_Id': int(input_.loc[a, 'Year_Id']),
                                        'Year': input_.loc[a, 'Year'],
                                        'Make_Id': [make['id'] for make in json_data['makes']],
                                        'Make': [make['value'] for make in json_data['makes']]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Year_Id': int(input_.loc[a, 'Year_Id']),
                                     'Year': input_.loc[a, 'Year']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Year'] + '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Year_Id', 'Make_Id'], ascending=[False, True]).reset_index(drop=True)
output_correct.to_excel('./make-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Year_Id'], ascending=[False]).reset_index(drop=True)
    output_error.to_excel('./make_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：47

[ok] - 2015
[尝试次数：1] - [剩余数量：37] - [当前时间：18:02:49]

[ok] - 2024
[尝试次数：1] - [剩余数量：36] - [当前时间：18:02:49]

[ok] - 2020
[尝试次数：1] - [剩余数量：35] - [当前时间：18:02:50]

[ok] - 2018
[尝试次数：1] - [剩余数量：34] - [当前时间：18:02:50]

[ok] - 2023
[尝试次数：1] - [剩余数量：33] - [当前时间：18:02:50]

[ok] - 2021
[尝试次数：1] - [剩余数量：32] - [当前时间：18:02:50]

[ok] - 2016
[尝试次数：1] - [剩余数量：31] - [当前时间：18:02:50]

[ok] - 2022
[尝试次数：1] - [剩余数量：30] - [当前时间：18:02:50]

[ok] - 2017
[尝试次数：1] - [剩余数量：29] - [当前时间：18:02:50]

[ok] - 2019
[尝试次数：1] - [剩余数量：28] - [当前时间：18:02:50]

[ok] - 2012
[尝试次数：1] - [剩余数量：27] - [当前时间：18:02:51]

[ok] - 2014
[尝试次数：1] - [剩余数量：26] - [当前时间：18:02:51]

[ok] - 2013
[尝试次数：1] - [剩余数量：25] - [当前时间：18:02:51]

[ok] - 2009
[尝试次数：1] - [剩余数量：24] - [当前时间：18:02:51]

[ok] - 2008
[尝试次数：1] - [剩余数量：23] - [当前时间：18:02:51]

[ok] - 2010
[尝试次数：1] - [剩余数量：22] - [当前时间：18:02:51]

[ok] - 2011
[尝试次数：1] - [剩余数量：21] - [当前时间：18:02:51]

[ok] - 2006
[尝试次数：1] - [剩余数量：20] - [当前时间：18:02:51]

[ok] - 2007
[尝试次数：1] - [剩余数量：19] - [当前时间：18:02:51]

[ok]